<a href="https://colab.research.google.com/github/DDiekmann/Applied-Verification-Lab-Neural-Networks/blob/main/Tutorials/Alpha_Beta_Crown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial for α,β-CROWN Robustness Verification 





---


*This tutorial shows the robustness verification of a neural network trained on 
the MNIST dataset with use of α,β-CROWN.*


---



**α,β-CROWN**
α,β-CROWN is an open-source neural network verifier. The code can be found on [their website](https://github.com/huanzhang12/alpha-beta-CROWN). 

In this tutorial we will first go through the installation of α,β-CROWN and train our own network. This network is to be verified later in the tutorial. Therefore, the verfication must be configured first. 

## Installation

This installation is based on another [tutorial](https://colab.research.google.com/drive/1mJTOmq2qHxMycHUzBepBN47QWcxda3ov#scrollTo=Y0toepwVIFTG). 

First we will set up our miniconda environment. 

In [1]:
%%capture
%%bash
%env PYTHONPATH=
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

Python in version 3.7 is installed into the environment. 

In [2]:
%%capture
%%bash
conda install --channel defaults conda python=3.7 --yes
conda update --channel defaults --all --yes

In [3]:
%%capture
import sys
sys.path
!ls /usr/local/lib/python3.7/dist-packages
_ = (sys.path
        .append("/usr/local/lib/python3.7/site-packages"))

In order to use the library, we have to clone the corresponding git-repository.

In [4]:
%%capture
!git clone https://github.com/huanzhang12/alpha-beta-CROWN.git

The environment is created. 

In [5]:
%%capture
%%bash
# Remove the old environment, if necessary.
conda env remove --name alpha-beta-crown
conda env create -f alpha-beta-CROWN/complete_verifier/environment.yml  # install all dependents into the alpha-beta-crown environment

In [6]:
%cd alpha-beta-CROWN/complete_verifier/

/content/alpha-beta-CROWN/complete_verifier


As explained on [the website of the α,β-project](https://github.com/huanzhang12/alpha-beta-CROWN), it is nessasary to create a configuration file in order to load the data. 

## Configure verification

Now we want to verifiy a model. Therefore, we create a file called my_example_config.yaml which configures parameters for verification. 

**TODO**: check parameters

In [77]:
%%writefile exp_configs/my_example_config.yaml
general:
  mode: verified-acc
  device: cpu
model:
  name: mnist_6_100
  path: models/eran/mnist_6_100_nat.pth
data:
  dataset: MNIST_ERAN_UN
  mean: [0.4914, 0.4822, 0.4465]
  std: [0.2023, 0.1994, 0.201]
  num_outputs: 10
specification:
  epsilon: 0.00784313725  # 2./255.
attack:
  pgd_restarts: 100
solver:
  beta-crown:
    batch_size: 2048
    iteration: 20
bab:
  max_domains: 5000000
  timeout: 300

Overwriting exp_configs/tutorial_cifar_example.yaml


## Verification of network with α,β-CROWN

Now we only have to run our verification. 

First, we activate our environment.
Then we call the robustness_verifier on our configured yaml-file. 
We finish by deactivating the environment.


In [78]:
%%bash
source activate alpha-beta-crown
python robustness_verifier.py --config exp_configs/my_example_config.yaml --start 3 --end 4
conda deactivate

Configurations:

general:
  device: cpu
  seed: 100
  conv_mode: patches
  deterministic: false
  double_fp: false
  loss_reduction_func: sum
  record_bounds: false
  mode: verified-acc
  complete_verifier: bab
  enable_incomplete_verification: true
  get_crown_verified_acc: false
model:
  path: models/eran/mnist_6_100_nat.pth
  name: mnist_6_100
data:
  start: 3
  end: 4
  num_outputs: 10
  mean: [0.4914, 0.4822, 0.4465]
  std: [0.2023, 0.1994, 0.201]
  pkl_path: null
  dataset: MNIST_ERAN_UN
  data_filter_path: null
  data_idx_file: null
specification:
  type: lp
  norm: .inf
  epsilon: 0.00784313725
solver:
  alpha-crown:
    lr_alpha: 0.1
    iteration: 100
    share_slopes: false
    no_joint_opt: false
  beta-crown:
    batch_size: 2048
    lr_alpha: 0.01
    lr_beta: 0.05
    lr_decay: 0.98
    optimizer: adam
    iteration: 20
    beta: true
    beta_warmup: true
  mip:
    parallel_solvers: null
    solver_threads: 1
    refine_neuron_timeout: 15
    refine_neuron_time_percent

/content/alpha-beta-CROWN/complete_verifier/utils.py:512: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  eps_temp = torch.tensor(eps_temp).reshape(1, -1, 1, 1)
